In [10]:
from  transformers  import XLMRobertaModel, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaModel.from_pretrained('xlm-roberta-base')
print('Model and Tokenizer Loaded.')





Model and Tokenizer Loaded.


In [57]:
import csv 
from  collections import namedtuple
from scipy.spatial import distance
import re 

def match_tokens(normal_tokens , model_tokens):
    nt_index = 0 
    mt_index = 1 
    ret = [] 
#     print ('Matching ' , normal_tokens, ' ' , model_tokens)
    while nt_index < len(normal_tokens) and mt_index < len(model_tokens): 
#         print (model_tokens[mt_index] , '<->' , normal_tokens[nt_index])
        if  model_tokens[mt_index].replace('▁' , '')  == normal_tokens[nt_index] :
            ret.append((nt_index , mt_index)) 
            nt_index += 1 
            mt_index += 1 
        elif  normal_tokens[nt_index].startswith(model_tokens[mt_index].replace('▁' , '')) or  model_tokens[mt_index].replace('▁' , '').startswith(normal_tokens[nt_index]): 
            ret.append((nt_index , mt_index)) 
            nt_index += 1 
            mt_index += 1 
        else:
            mt_index += 1 
    return ret  

def load_wic_data(file_name , is_train=True):
    DataModel = namedtuple('DataModel' , 'word sent_a sent_b is_same')
    ret = [] 
    with open(file_name,  'r' , encoding='utf8') as f: 
        data_file = csv.reader(f , delimiter='\t') 
        for row in data_file:
            if is_train: 
                ret.append(DataModel(row[0], row[6].replace('\u200c' , ' ') , row[7].replace('\u200c' , ' ') , row[8]))
            else:
                ret.append(DataModel(row[0], row[6].replace('\u200c' , ' ') , row[7].replace('\u200c' , ' ') , '0'))
    return ret 


def classify(train, model , tokenizer , threshold): 
    success = 0 
    for index,data in enumerate(train): 
        sent_a_tokens = data.sent_a.split(' ')
        sent_b_tokens = data.sent_b.split(' ')
        tokens_a = tokenizer(data.sent_a)
        tokens_b = tokenizer(data.sent_b)
        input_ids_a = torch.tensor(tokens_a['input_ids']).unsqueeze(0)  
        input_ids_b = torch.tensor(tokens_b['input_ids']).unsqueeze(0)  
        model_tokens_a = tokenizer.convert_ids_to_tokens(tokens_a['input_ids']) 
        model_tokens_b = tokenizer.convert_ids_to_tokens(tokens_b['input_ids']) 

        outputs_a = model(input_ids_a)        
        outputs_b = model(input_ids_b)
        
        match_a = match_tokens(sent_a_tokens , model_tokens_a)
        match_b = match_tokens(sent_b_tokens , model_tokens_b)
        
        
#         print(data)
#         print(model_tokens_a)
#         print(sent_a_tokens)
#         print(model_tokens_b)
#         print(sent_b_tokens)
        ind_a = 0 
        ind_b = 0 
        
        for x,y in match_a: 
#             print('a' , sent_a_tokens[x])
            if sent_a_tokens[x].startswith(data.word) or data.word.startswith(sent_a_tokens[x]) : 
                break 
            ind_a += 1 
        
        for x,y in match_b: 
#             print('b' , sent_b_tokens[x])
            if sent_b_tokens[x].startswith(data.word) or data.word.startswith(sent_b_tokens[x]) : 
                break 
            ind_b += 1 

    
        if ind_a >= len(match_a) or ind_b >= len(match_b): 
            print('Cannot find match {}  in data {}'.format(data.word , data))
            break
            continue 
        
        diff = distance.cosine(outputs_a[0][0, ind_a , : ].detach().numpy(),  outputs_b[0][0, ind_b , : ].detach().numpy())
        predict = '0'
        if diff < threshold: 
            predict = '1'
        
        if predict == data.is_same:
            success += 1 
        
        
        
            
    print('Success Rate {}'.format(success / len(train)))
        

In [58]:
train = load_wic_data('../data/wic/fa/valid.txt') 
classify(train , model, tokenizer, 0.5)

Success Rate 0.505
